In [1]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [2]:
def load_data(hdf5_file, dataset_name_images, dataset_name_labels):
    with h5py.File(hdf5_file, 'r') as hf:
        data = np.array(hf[dataset_name_images])
        labels = np.array(hf[dataset_name_labels])
    return data, labels

# Loading model and Testing accuracy

In [3]:
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Reshape, Dense, Multiply, Conv2D, Add, Activation, BatchNormalization
from tensorflow.keras.applications import MobileNetV3Small

In [4]:
# Channel Attention Module
class ChannelAttention(Layer):
    def __init__(self, reduction_ratio=16, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.global_avg_pool = GlobalAveragePooling2D()
        self.reshape = Reshape((1, 1, input_shape[-1]))
        self.dense1 = Dense(input_shape[-1] // self.reduction_ratio, activation='relu', use_bias=False)
        self.dense2 = Dense(input_shape[-1], activation='sigmoid', use_bias=False)

    def call(self, inputs):
        x = self.global_avg_pool(inputs)
        x = self.reshape(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return Multiply()([inputs, x])


# Spatial Attention Module
class SpatialAttention(Layer):
    def __init__(self, kernel_size=7, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.conv = Conv2D(1, kernel_size, padding='same', activation='sigmoid')

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        return Multiply()([inputs, self.conv(concat)])

# CBAM Module
class CBAM(Layer):
    def __init__(self, reduction_ratio=16, kernel_size=7, **kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.channel_attention = ChannelAttention(reduction_ratio=reduction_ratio)
        self.spatial_attention = SpatialAttention(kernel_size=kernel_size)

    def call(self, inputs):
        x = self.channel_attention(inputs)
        x = self.spatial_attention(x)
        return x

In [5]:
LEC_loaded_model = tf.keras.models.load_model('Trained Model/LEC_model.keras')
LEC_loaded_model1 = tf.keras.models.load_model('Trained Model/LEC_model2.keras')
LEC_no_FCL = tf.keras.models.load_model('Trained Model/LEC_model_no_FCL.keras')
LEC_loaded_model2 = tf.keras.models.load_model('Trained Model/LEC_model_CBAM.keras', custom_objects={"CBAM": CBAM})
LEC_loaded_model3 = tf.keras.models.load_model('Trained Model/LEC_model_Channel.keras', custom_objects={"ChannelAttention": ChannelAttention})
LEC_loaded_model4 = tf.keras.models.load_model('Trained Model/LEC_model_with_layers.keras')
LEC_loaded_model5 = tf.keras.models.load_model('Trained Model/LEC_model_general.keras')
LEC_loaded_model6 = tf.keras.models.load_model('Trained Model/LEC_model_no_TL.keras')
MobileNetV3_S = tf.keras.models.load_model('Trained Model/MobileNetV3Small.keras')
MobileNetV3_S2 = tf.keras.models.load_model('Trained Model/MobileNetV3Small_no_TL.keras')
MobileNetV3_S3 = tf.keras.models.load_model('Trained Model/MobileNetV3Small_with_FCL.keras')
MobileNetV3_L = tf.keras.models.load_model('Trained Model/Teacher_MobileNetV3Large_model.keras')




e:\AI Researches\Light Emotion CNN\Code\myenv\lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'cbam_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [6]:
LEC_loaded_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_no_FCL.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model3.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model4.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model5.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
LEC_loaded_model6.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
X_fer_test, y_fer_test = load_data('output files/fer2013_testing.h5', 'images', 'labels')
X_rafdb_test, y_rafdb_test = load_data('output files/rafdb_testing.h5', 'images', 'labels')
X_ck_test, y_ck_test = load_data('output files/rafdb_testing.h5', 'images', 'labels')

## Assessing Accuracy on FER-2013

In [8]:
score = LEC_loaded_model.evaluate(X_fer_test, y_fer_test, verbose=1)
score = LEC_loaded_model1.evaluate(X_fer_test, y_fer_test, verbose=1)
score = LEC_no_FCL.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = LEC_loaded_model2.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = LEC_loaded_model3.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = LEC_loaded_model4.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = LEC_loaded_model5.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = LEC_loaded_model6.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = MobileNetV3_S.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = MobileNetV3_S2.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = MobileNetV3_S3.evaluate(X_fer_test, y_fer_test, verbose=1)
# score = MobileNetV3_L.evaluate(X_fer_test, y_fer_test, verbose=1)



225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - accuracy: 0.6218 - loss: 1.1418
225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 86ms/step - accuracy: 0.6282 - loss: 1.2469
225/225 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.6236 - loss: 1.3806
